In [23]:
import os

os.environ['KALDI_ROOT'] = '/home/faber6911/kaldi/'

import kaldi_io
import numpy as np
import librosa
import IPython.display as ipd
import random
import kaldiio
from kaldiio import WriteHelper
from utils import add_noise

In [24]:
ark_path_train = '/home/faber6911/kaldi/egs/Signal-denoising-in-the-wild/data/train/train.ark'
scp_path_train = '/home/faber6911/kaldi/egs/Signal-denoising-in-the-wild/data/train/train.scp'
ark_path_test = '/home/faber6911/kaldi/egs/Signal-denoising-in-the-wild/data/test/noisy_audio.ark'
scp_path_test = '/home/faber6911/kaldi/egs/Signal-denoising-in-the-wild/data/test/noisy_audio.scp'
train_augmentation = 1
test_augmentation = 1

In [25]:
for line in open('../data/train/wav.scp'):
    print(line)
    break

EN_C1_12_107 /export/corpora/siwis_database/wav/EN/EN_C1_12/EN_C1_12_107.wav



In [26]:
for line in open('../data/utt2spk.scp'):
    print(line)
    break

DE_A1_03_000 03



In [4]:
uno = np.array([1,2,3,4,5])
dos = np.array([6,7,8,9,10])

In [10]:
uno = uno.reshape(1, -1)
dos = dos.reshape(1, -1)

In [16]:
tres = np.concatenate((uno, dos), axis = 0)

In [17]:
prova, prova2 = tres

In [19]:
prova, prova2

(array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10]))

In [42]:
print('Adding noise to train audio and augmenting each file {} times'.format(train_augmentation*3))
writer = WriteHelper('ark,scp:{},{}'.format(ark_path_train, scp_path_train), compression_method=2)

noise_choice = {'music':659, 'noise':929, 'speech':425}

for count, line in enumerate(open('../data/train/wav.scp')):
    # clean audio path
    utt, path = line.rstrip().split()
    # clean audio file
    clean_audio, _ = librosa.load(path, sr = 16000)
    # now for every noise type we augment n times the clean audio file using random noise audio files
    for noise_type in noise_choice:
        for aug in range(train_augmentation):
            noise_track = np.random.randint(0, noise_choice[noise_type])
            _, noise_path = open('../data/musan_{}.scp'.format(noise_type)).readlines()[noise_track].rstrip().split()
            noise_audio, _ = librosa.load(noise_path, sr = 16000)
            noisy_audio = add_noise(clean_audio, noise_audio, snr=random.choice([2.5, 7.5, 12.5, 17.5]))
            # write ark and associated scp file in train directory
            writer(utt, np.concatenate((clean_audio.reshape(1, -1), noisy_audio.reshape(1, -1))))
    if count % 10 == 0:
        break
        #print('Augmented {} audio files'.format(count))
    
writer.close()
print('Augmented train data')

Adding noise to train audio and augmenting each file 3 times
Augmented train data


In [32]:
for line in open('../data/train/train.scp'):
    utt, path = line.rstrip().split()
    clean, noisy = kaldi_io.read_mat(path)
    break

In [36]:
ipd.Audio(clean, rate = 16000)

In [37]:
ipd.Audio(noisy, rate = 16000)

In [38]:
print('Adding noise to test audio and augmenting each file {} times'.format(test_augmentation*3))
writer = WriteHelper('ark,scp:{},{}'.format(ark_path_test, scp_path_test), compression_method=1)

noise_choice = {'music':659, 'noise':929, 'speech':425}

for count, line in enumerate(open('../data/test/wav.scp')):
    # clean audio path
    utt, path = line.rstrip().split()
    # clean audio file
    clean_audio, _ = librosa.load(path, sr = 16000)
    # now for every noise type we augment n times the clean audio file using random noise audio files
    for noise_type in noise_choice:
        for aug in range(test_augmentation):
            noise_track = np.random.randint(0, noise_choice[noise_type])
            _, noise_path = open('../data/musan_{}.scp'.format(noise_type)).readlines()[noise_track].rstrip().split()
            noise_audio, _ = librosa.load(noise_path, sr = 16000)
            noisy_audio = add_noise(clean_audio, noise_audio, snr=random.choice([2.5, 7.5, 12.5, 17.5]))
            # write ark and associated scp file in train directory
            writer(path, noisy_audio.reshape(-1, 1))
    if count % 100 == 0:
        print('Augmented {} audio files'.format(count))
    
writer.close()
print('Augmented test data')

Adding noise to test audio and augmenting each file 3 times
Augmented 0 audio files


KeyboardInterrupt: 